# Counter Strikle Webscraper

## Notebook Description

__Author:__ Daniël Vermaas

This notebook scrapes [Liquidpedia](https://liquipedia.net/counterstrike/Main_Page), in order to make [Counter-Strikle](https://blast.tv/counter-strikle) puzzle-solving easier. Before using the notebook, please read the [Liquidpedia ToS about API usage](https://liquipedia.net/api-terms-of-use).

## Libraries & Constants

In [1]:
import os
import time
from bs4 import BeautifulSoup
from datetime import datetime
import json
import mwparserfromhell
from ratelimit import limits, sleep_and_retry
import pandas as pd
from tqdm.notebook import tqdm
import requests
import ipywidgets as widgets
from ipywidgets import interact

In [2]:
BASE_URL = "https://liquipedia.net/counterstrike/api.php?"
HEADERS = {"User-Agent": "Counter-Strikle-Bot (dvermaas@live.com)","Accept-Encoding": "gzip"}
CSV_FILE = "players.csv"
QUERY_COOLDOWN = 2
REGIONS = {
    "Europe" : ["Europe", "CIS"],
    "Americas" : ["North America", "South America"],
    "Asia-Pacific" : ["Oceania", "Asia"],
}
COLUMNS = [
    "PAGE",
    "NAME",
    "REAL NAME",
    "REGION",
    "NATIONALITY",
    "TEAM",
    "AGE",
    "WEAPON",
    "MAJOR APPEARANCES",
    "LAST UPDATED"
]

## MediaWiki API requests

In [3]:
@sleep_and_retry
@limits(calls=1, period=30)
def fetch_page(page):
    url =  f"{BASE_URL}action=parse&format=json&page={page}"
    response = requests.get(url, HEADERS)
    page_html = response.json()['parse']['text']['*']
    soup = BeautifulSoup(page_html,features="lxml")
    return soup

@sleep_and_retry
@limits(calls=1, period=QUERY_COOLDOWN)
def query_page(page, rvsection):
    params = {
        "action": "query",
        "format": "json",
        "prop": "revisions",
        "titles": page,
        "rvprop": "content",
        "rvslots": "*",
        "rvsection": rvsection,
    }
    response = requests.get(BASE_URL, params=params, headers=HEADERS)
    data = response.json()
    pages = data["query"]["pages"]
    page_id = next(iter(pages))
    page = pages[page_id]
    wikicode = mwparserfromhell.parse(page["revisions"][0]["slots"]["main"]["*"])
    return wikicode

## Generate Country to Region Map

In [4]:
def build_region_dict():
    country_region_dict = {}
    with open("regions.json", "r") as file:
        region_dict = json.load(file)
    for region, countries in region_dict.items():
            for country in countries:
                country_region_dict[country] = region
    return country_region_dict

REGION_DICT = build_region_dict()

In [5]:
# Fetches list of all concluded majors (checks hltv awards) 
def fetch_majors():
    wikicode = query_page("Majors", 8)
    links = wikicode.filter_wikilinks()
    links = [link.split('|')[0].replace('[[', '') for link in links]
    return links

MAJOR_LIST = fetch_majors()

In [6]:
# Fetches all player ids
def fetch_players():
    results = []
    for category in ["Players", "Coaches"]:
        url = f"{BASE_URL}action=query&list=categorymembers&cmtitle=Category:{category}&cmlimit=max&format=json"

        while True:
            response = requests.get(url)
            time.sleep(QUERY_COOLDOWN)
            data = json.loads(response.text)
            pages = data["query"]["categorymembers"]
            results.extend(pages)

            if "continue" not in data:
                break

            cont = data["continue"]
            cmcontinue = cont["cmcontinue"]
            url = f"{url}&cmcontinue={cmcontinue}"
    results = [row["title"] for row in results if "Category" not in row["title"]]
    return results

PLAYER_LIST = fetch_players()

In [7]:
def fetch_major_players():
    # load majors from json
    file_name = "majors.json"
    with open(file_name, "r") as file:
        major_dict = json.load(file)

    # check for new majors
    for major in fetch_majors():
        if major in major_dict:
            continue
        wikicode = query_page(major, "5")
        teams_dict = {}
        for template in wikicode.filter_templates(matches=r"\bTeamCard\b"):
            if template.name.strip() == "TeamCard":
                # skip showmatches
                if not template.has("qualifier"):
                    continue
                if not template.has("p1"):
                    continue
                team = template.get("team").value.strip()
                players = [template.get(f"p{i}").value.strip() for i in range(1, 6)]
                teams_dict[team] = players
        major_dict[major] = teams_dict

    # save majors to json
    with open(file_name, "w") as file:
        json.dump(major_dict, file, indent=2)

    # invert dict
    major_players = {}
    for major,teams in major_dict.items():
        for team, players in teams.items():
            for player in players:
                if player not in major_players:
                    major_players[player] = 0
                major_players[player] += 1

    return major_players

MAJOR_PLAYERS = fetch_major_players()

## Player-Specific Information

In [8]:
# fetches all player data and puts it in dict
def fetch_player_dict(player_id):
    infobox_dict = {}
    wikicode = query_page(player_id, 0)
    try:
        infobox = wikicode.filter_templates(matches="Infobox player")[0]
    except IndexError:
        return infobox_dict
    for param in infobox.params:
        value = mwparserfromhell.parse(param.value.strip_code()).strip()
        infobox_dict[param.name.strip()] = str(value)
    return infobox_dict

# filter player data for relevant information
def fetch_player(player_id):
    # parse player information
    info_dict = fetch_player_dict(player_id)

    # add name to dict
    output_dict = dict()
    output_dict["PAGE"] = player_id
    output_dict["NAME"] = info_dict.get("id")
    output_dict["REGION"] = REGION_DICT.get(info_dict.get("country"))
    output_dict["REAL NAME"] = info_dict.get("romanized_name", info_dict.get("name"))
    output_dict["NATIONALITY"] = info_dict.get("country")
    output_dict["TEAM"] = info_dict.get("team")
    output_dict["AGE"] = info_dict.get("birth_date")
    roles = [info_dict.get("role"), info_dict.get("role2")]
    if "awp" in roles:
        output_dict["WEAPON"] = "AWP"
    else:
        output_dict["WEAPON"] = "AK47"
    output_dict["MAJOR APPEARANCES"] = MAJOR_PLAYERS.get(info_dict.get("id"), 0)
    output_dict["LAST UPDATED"] = datetime.now().strftime("%Y-%m-%d")
    return output_dict

fetch_player("Boomser")

{'PAGE': 'Boomser',
 'NAME': None,
 'REGION': None,
 'REAL NAME': None,
 'NATIONALITY': None,
 'TEAM': None,
 'AGE': None,
 'WEAPON': 'AK47',
 'MAJOR APPEARANCES': 0,
 'LAST UPDATED': '2023-05-19'}

## Generate CSV

In [9]:
def build_df():
    # Get dataframe to write to
    if os.path.exists(CSV_FILE):
        playerdata = pd.read_csv(CSV_FILE)
    else:
        playerdata = pd.DataFrame(columns=COLUMNS)

    try:
        for player in tqdm(PLAYER_LIST, leave=False):
            if not (playerdata["PAGE"].eq(player)).any():
                row_data = fetch_player(player)
                playerdata = pd.concat([playerdata, pd.DataFrame.from_records([row_data])], ignore_index=True)
    except Exception as e:
        print("Error:", e)
        print("Player:", player)
    
    playerdata.to_csv(CSV_FILE, index=False)

build_df()

  0%|          | 0/3573 [00:00<?, ?it/s]

In [36]:
pd.read_csv(CSV_FILE)

,PAGE,NAME,REAL NAME,REGION,NATIONALITY,TEAM,AGE,WEAPON,MAJOR APPEARANCES,LAST UPDATED
0,-Ace,-Ace,Brandon Winn,Americas,United States,NaN,1995-4-7,AK47,0,2023-05-19
1,.exe,.exe,Aidan Vorster,NaN,South Africa,Goliath Gaming,2003-5-15,AK47,0,2023-05-19
2,.PhP,.PhP,Marek Kádek,NaN,Slovakia,NaN,NaN,AK47,0,2023-05-19
3,0i,0i,Liu Zhiwei,Asia-Pacific,China,Invictus Gaming,1996-06-18,AK47,0,2023-05-19
4,129,129,Yang Ting,Asia-Pacific,China,NaN,NaN,AK47,0,2023-05-19
...,...,...,...,...,...,...,...,...,...,...
3536,Zews,zews,Wilton Prado,Americas,Brazil,Fluxo,1987-11-3,AK47,1,2023-05-19
3537,ZoneR,zoneR,Aleksandr Anatolyevich Bogatyrev,Europe,Russia,forZe School,1987-08-04,AK47,0,2023-05-19
3538,Zonic,zonic,Danny Selim Sørensen,Europe,Denmark,Team Vitality,1986-7-30,AK47,0,2023-05-19
3539,Zote,Zote,Julian Acosta,Americas,Argentina,BESTIA,NaN,AK47,0,2023-05-19


## Post Processing

In [35]:
def age(birthdate):
    birthdate = datetime.strptime(birthdate, "%Y-%m-%d")
    today = datetime.now()
    return today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))

def playerdata_postprocess(df):
    columns = list(pd.read_csv(CSV_FILE).columns)
    del columns[columns.index("TEAM")]
    df.dropna(inplace = True, subset = columns)
    age_conversions = []
    for date in df["AGE"]:
        try:
            age_conversions.append(age(date))
        except ValueError:
            age_conversions.append(0)
    df["AGE"] = age_conversions
    return df.sort_values(["MAJOR APPEARANCES"], ascending=False)

playerdata_post = playerdata_postprocess(pd.read_csv(CSV_FILE))

,PAGE,NAME,REAL NAME,REGION,NATIONALITY,TEAM,AGE,WEAPON,MAJOR APPEARANCES,LAST UPDATED
710,Dupreeh,dupreeh,Peter Rothmann Rasmussen,Europe,Denmark,Team Vitality,30,AWP,17,2023-05-19
3035,Xyp9x,Xyp9x,Andreas Højsleth,Europe,Denmark,Astralis Talent,27,AK47,16,2023-05-19
2497,Shox,shox,Richard Papillon,Europe,France,NaN,30,AK47,16,2023-05-19
126,ApEX,apEX,Dan Madesclaire,Europe,France,Team Vitality,30,AK47,15,2023-05-19
1354,Karrigan,karrigan,Finn Andersen,Europe,Denmark,FaZe Clan,33,AK47,15,2023-05-19
...,...,...,...,...,...,...,...,...,...,...
1298,Juissi,juissi,Youssef Adam,Europe,Finland,ENCE Academy,20,AK47,0,2023-05-19
1299,Juli,Juli,Juliana Tosic,Americas,United States,NaN,23,AK47,0,2023-05-19
1300,Juliano,juliano,Julia Kiran,Europe,Sweden,G2 Oya,29,AK47,0,2023-05-19
1302,Julih,julih,Julia Gomes,Americas,Brazil,B4 Esports Female,24,AK47,0,2023-05-19


## UI Search Engine

In [34]:
def generate_selection(data, description):
    unique_data = list(data.unique())
    return widgets.SelectMultiple(
        options=unique_data,
        value=unique_data,
        rows=len(unique_data),
        description=description,
        disabled=False)


def generate_rangeslider(data, description):
    return widgets.FloatRangeSlider(
        value=[min(data), max(data)],
        min=min(data),
        max=max(data),
        description=description,
        step=1,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='1',
)

region_selector = generate_selection(playerdata_post["REGION"], "REGION")
age_slider = generate_rangeslider(playerdata_post["AGE"], "AGE")
weapon_selector = generate_selection(playerdata_post["WEAPON"], "WEAPON")
majors_slider = generate_rangeslider(playerdata_post["MAJOR APPEARANCES"], "MAJOR APPEARANCES")

def ui_selector(sregion, sage, sweapon, smajors):
    return playerdata_post.loc[(playerdata_post["REGION"].isin(sregion)) &
                               (playerdata_post["AGE"] >= sage[0]) &
                               (playerdata_post["AGE"] <= sage[1]) &
                               (playerdata_post["WEAPON"].isin(sweapon)) &
                               (playerdata_post["MAJOR APPEARANCES"] >= smajors[0]) &
                               (playerdata_post["MAJOR APPEARANCES"] <= smajors[1])
                               ]
    
interact(ui_selector, sregion = region_selector, sage = age_slider, sweapon = weapon_selector, smajors = majors_slider)

interactive(children=(SelectMultiple(description='REGION', index=(0, 1, 2), options=('Europe', 'Americas', 'As…

<function __main__.ui_selector(sregion, sage, sweapon, smajors)>

In [42]:
from selenium import webdriver

# Configure ChromeOptions to remove webdriver flag and disable automation features
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')

# Provide the path to the ChromeDriver executable
chromedriver_path = 'chromedriver.exe'

# Open the Chrome browser with the specified options
browser = webdriver.Chrome(executable_path=chromedriver_path, options=options)


browser.get("https://blast.tv")
time.sleep(5)
browser.find_element_by_xpath("//button[@class='coi-consent-banner__agree-button']").click()
time.sleep(1)
browser.get("https://blast.tv/counter-strike")
search_bar = browser.find_element_by_xpath("//input[@id='player-search']")
search_bar.send_keys("-")
time.sleep(1)

# Locate the autocomplete suggestion elements
suggestions = browser.find_elements_by_xpath("//ul[@id='player-search-autocomplete']/li")
print(suggestions)
# Close the browser instance
browser.quit()

C:\Users\dverm\AppData\Local\Temp\ipykernel_13472\352293243.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=chromedriver_path, options=options)


AttributeError: 'WebDriver' object has no attribute 'find_element_by_xpath'

In [19]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configure ChromeOptions to remove webdriver flag and disable automation features
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')

browser = webdriver.Chrome(options=options)
browser.get("https://blast.tv/counter-strikle")

cookies_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='coi-consent-banner__agree-button']")))
cookies_button.click()

rules_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='styled__StyledButton-sc-y8jf4t-0 eccxcB']")))
rules_button.click()

search_bar = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@class='styled__SearchBoxInput-sc-1uwunev-2 uWYFV']")))
search_bar.send_keys('b')
time.sleep(2)
autocomplete_results = browser.find_elements(By.XPATH, "//div[contains(@class, 'styled__SearchResultRow-sc-1uwunev-4')]")
autocomplete_texts = [result.text for result in autocomplete_results]

In [16]:
print(len(autocomplete_texts), autocomplete_texts)

31 ['Boombl4 - Kirill Mikhailov', 'b1t - Valeriy Vakhovskiy', 'byali - Paweł Bieliński', 'Brehze - Vincent Cayonte', 'buster - Timur Tulepov', 'boltz - Ricardo Prass', 'broky - Helvijs Saukants', 'bodyy - Alexandre Pianaro', 'Bubzkji - Lucas Andersen', 'Brollan - Ludvig Brolin', 'bubble - Kamen Kostadinov', 'blameF - Benjamin Bremer', 'BnTeT - Hansel Ferdinand', 'b0RUP - Johannes Borup', 'bondik - Vladyslav Nechyporchuk', 'blocker - Yanko Panov', 'Bymas - Aurimas Pipiras', 'BLACKEAGLE - Moustafa Baghdadi', 'B1ad3 - Andrey Gorodenskiy', 'ben1337 - Ben Smith', 'BARBARR - André Möller', 'BIT - Bruno Fukuda Lima', 'bottle - Lei Mao', 'beastik - Sebastian Daňo', 'birdfromsky - Thomas Due-Frederiksen', 'Benkai - Benedict Tan', 'b4rtiN - Bruno Câmara', 'boX - Anton Burko', 'biguzera - Rodrigo Bittencourt', 'Bååten - Linus Andersson', 'brnz4n - Breno Poletto']
